# Import datasets taken from MOH

In [1]:
import pandas as pd

In [2]:
cases_df = pd.read_csv("cases_state.csv")
cases_df.head()

date            state  cases_new  cases_import  cases_recovered  \
0  2020-01-25            Johor          4             4                0   
1  2020-01-25            Kedah          0             0                0   
2  2020-01-25         Kelantan          0             0                0   
3  2020-01-25           Melaka          0             0                0   
4  2020-01-25  Negeri Sembilan          0             0                0   

   cases_active  cases_cluster  cases_unvax  cases_pvax  cases_fvax  ...  \
0             4              0            4           0           0  ...   
1             0              0            0           0           0  ...   
2             0              0            0           0           0  ...   
3             0              0            0           0           0  ...   
4             0              0            0           0           0  ...   

   cases_0_4  cases_5_11  cases_12_17  cases_18_29  cases_30_39  cases_40_49  \
0          0           0            0            0            0            1   
1          0           0            0            0            0            0   
2          0           0            0            0            0            0   
3          0           0            0            0            0            0   
4          0           0            0            0            0            0   

   cases_50_59  cases_60_69  cases_70_79  cases_80  
0            0            0            0         0  
1            0            0            0         0  
2            0            0            0         0  
3            0            0            0         0  
4            0            0            0         0  

[5 rows x 25 columns]

In [3]:
deaths_df = pd.read_csv("deaths_state.csv")
deaths_df.head()

date            state  deaths_new  deaths_bid  deaths_new_dod  \
0  2020-03-17            Johor           1           0               1   
1  2020-03-17            Kedah           0           0               0   
2  2020-03-17         Kelantan           0           0               0   
3  2020-03-17           Melaka           0           0               0   
4  2020-03-17  Negeri Sembilan           0           0               0   

   deaths_bid_dod  deaths_unvax  deaths_pvax  deaths_fvax  deaths_boost  \
0               0             1            0            0             0   
1               0             0            0            0             0   
2               0             0            0            0             0   
3               0             0            0            0             0   
4               0             0            0            0             0   

   deaths_tat  
0           0  
1           0  
2           0  
3           0  
4           0

In [4]:
beds_df = pd.read_csv("hospital.csv")
beds_df.head()

date            state  beds  beds_covid  beds_noncrit  admitted_pui  \
0  2020-03-24            Johor  1440         434          1408             0   
1  2020-03-24            Kedah  1218         143          1183             0   
2  2020-03-24         Kelantan  1463         280          1424             9   
3  2020-03-24           Melaka  1091          82          1065             0   
4  2020-03-24  Negeri Sembilan  1223         253          1205             0   

   admitted_covid  admitted_total  discharged_pui  discharged_covid  \
0               0               0               0                 0   
1               0               0               0                 0   
2               6              15               1                 0   
3               0               0               0                 0   
4               0               0               0                 0   

   discharged_total  hosp_covid  hosp_pui  hosp_noncovid  
0                 0          18         9              1  
1                 0          46         5             12  
2                 1          78        58             23  
3                 0          19        10             12  
4                 0          90        13              8

In [5]:
icu_df = pd.read_csv("icu.csv")
icu_df.head()

date            state  beds_icu  beds_icu_rep  beds_icu_total  \
0  2020-03-24            Johor        10            22              32   
1  2020-03-24            Kedah        35             0              35   
2  2020-03-24         Kelantan        39             0              39   
3  2020-03-24           Melaka        26             0              26   
4  2020-03-24  Negeri Sembilan        18             0              18   

   beds_icu_covid  vent  vent_port  icu_covid  icu_pui  icu_noncovid  \
0              10    11          2          0        0             1   
1              16    34          0          2        1            12   
2              20    42          4          1        0            20   
3              12    23          4          0        2            12   
4              10    14          9          0        0             8   

   vent_covid  vent_pui  vent_noncovid  vent_used  vent_port_used  
0           0         0              0          8               0  
1           1         1              0         19               0  
2           1         0              0         26               0  
3           0         1              0         13               5  
4           0         0              0          1               6

Drop extra columns and convert date to datetime

In [6]:
cases_df = cases_df.loc[:, cases_df.columns.intersection(['date','state', 'cases_new'])]
cases_df['date'] = pd.to_datetime(cases_df['date'])
cases_df.head()

date            state  cases_new
0 2020-01-25            Johor          4
1 2020-01-25            Kedah          0
2 2020-01-25         Kelantan          0
3 2020-01-25           Melaka          0
4 2020-01-25  Negeri Sembilan          0

In [7]:
deaths_df = deaths_df.loc[:, deaths_df.columns.intersection(['date','state', 'deaths_new'])]
deaths_df['date'] = pd.to_datetime(deaths_df['date'])
deaths_df.head()

date            state  deaths_new
0 2020-03-17            Johor           1
1 2020-03-17            Kedah           0
2 2020-03-17         Kelantan           0
3 2020-03-17           Melaka           0
4 2020-03-17  Negeri Sembilan           0

In [8]:
beds_df = pd.merge(beds_df, icu_df, on=['date','state'])
beds_df = beds_df.loc[:, beds_df.columns.intersection(['date','state', 'beds', 'beds_covid', 'beds_icu_covid'])]
beds_df['date'] = pd.to_datetime(beds_df['date'])
beds_df.head()

date            state  beds  beds_covid  beds_icu_covid
0 2020-03-24            Johor  1440         434              10
1 2020-03-24            Kedah  1218         143              16
2 2020-03-24         Kelantan  1463         280              20
3 2020-03-24           Melaka  1091          82              12
4 2020-03-24  Negeri Sembilan  1223         253              10

# Combine datasets

In [9]:
from datetime import date, timedelta, datetime
# format: y/m/d
sdate = date(2020,1,1)
edate = date(2022,2,15)

states = list(cases_df['state'].unique())

new_df=pd.DataFrame(columns=['date','state', 'cases_new', 'deaths_new', 'beds', 'beds_covid', 'beds_icu_covid'])
row_count = 0
for date in pd.date_range(sdate,edate-timedelta(days=1),freq='d'):
    for state in states:
        row_count += 1
        new_df.loc[row_count, 'date'] = datetime.strftime(date, '%Y-%m-%d')
        new_df.loc[row_count, 'state'] = state
# new_df['date'] = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
new_df.head(20)

date              state cases_new deaths_new beds beds_covid  \
1   2020-01-01              Johor       NaN        NaN  NaN        NaN   
2   2020-01-01              Kedah       NaN        NaN  NaN        NaN   
3   2020-01-01           Kelantan       NaN        NaN  NaN        NaN   
4   2020-01-01             Melaka       NaN        NaN  NaN        NaN   
5   2020-01-01    Negeri Sembilan       NaN        NaN  NaN        NaN   
6   2020-01-01             Pahang       NaN        NaN  NaN        NaN   
7   2020-01-01              Perak       NaN        NaN  NaN        NaN   
8   2020-01-01             Perlis       NaN        NaN  NaN        NaN   
9   2020-01-01       Pulau Pinang       NaN        NaN  NaN        NaN   
10  2020-01-01              Sabah       NaN        NaN  NaN        NaN   
11  2020-01-01            Sarawak       NaN        NaN  NaN        NaN   
12  2020-01-01           Selangor       NaN        NaN  NaN        NaN   
13  2020-01-01         Terengganu       NaN        NaN  NaN        NaN   
14  2020-01-01  W.P. Kuala Lumpur       NaN        NaN  NaN        NaN   
15  2020-01-01        W.P. Labuan       NaN        NaN  NaN        NaN   
16  2020-01-01     W.P. Putrajaya       NaN        NaN  NaN        NaN   
17  2020-01-02              Johor       NaN        NaN  NaN        NaN   
18  2020-01-02              Kedah       NaN        NaN  NaN        NaN   
19  2020-01-02           Kelantan       NaN        NaN  NaN        NaN   
20  2020-01-02             Melaka       NaN        NaN  NaN        NaN   

   beds_icu_covid  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
5             NaN  
6             NaN  
7             NaN  
8             NaN  
9             NaN  
10            NaN  
11            NaN  
12            NaN  
13            NaN  
14            NaN  
15            NaN  
16            NaN  
17            NaN  
18            NaN  
19            NaN  
20            NaN

Take data from old datasets into new datasets.

In [10]:
import datetime
def transpose(new_df, old_df, cols):
    for index, row in new_df.iterrows():
        current_date = new_df.loc[index, 'date']
        current_state = new_df.loc[index, 'state']
        for index1, row1 in old_df.loc[old_df['date'] == current_date].iterrows():
            state = old_df.loc[index1, 'state']
            if current_state == state:
                for col in cols:
                    new_df.loc[index, col] = old_df.loc[index1, col]

transpose(new_df, cases_df, ['cases_new'])
new_df.tail(20)

date              state cases_new deaths_new beds beds_covid  \
12397  2022-02-13         Terengganu       297        NaN  NaN        NaN   
12398  2022-02-13  W.P. Kuala Lumpur      1001        NaN  NaN        NaN   
12399  2022-02-13        W.P. Labuan       109        NaN  NaN        NaN   
12400  2022-02-13     W.P. Putrajaya       105        NaN  NaN        NaN   
12401  2022-02-14              Johor      2800        NaN  NaN        NaN   
12402  2022-02-14              Kedah      2733        NaN  NaN        NaN   
12403  2022-02-14           Kelantan      1330        NaN  NaN        NaN   
12404  2022-02-14             Melaka       666        NaN  NaN        NaN   
12405  2022-02-14    Negeri Sembilan       707        NaN  NaN        NaN   
12406  2022-02-14             Pahang       938        NaN  NaN        NaN   
12407  2022-02-14              Perak       461        NaN  NaN        NaN   
12408  2022-02-14             Perlis       129        NaN  NaN        NaN   
12409  2022-02-14       Pulau Pinang      1656        NaN  NaN        NaN   
12410  2022-02-14              Sabah      4035        NaN  NaN        NaN   
12411  2022-02-14            Sarawak       206        NaN  NaN        NaN   
12412  2022-02-14           Selangor      3955        NaN  NaN        NaN   
12413  2022-02-14         Terengganu       376        NaN  NaN        NaN   
12414  2022-02-14  W.P. Kuala Lumpur      1010        NaN  NaN        NaN   
12415  2022-02-14        W.P. Labuan       189        NaN  NaN        NaN   
12416  2022-02-14     W.P. Putrajaya       124        NaN  NaN        NaN   

      beds_icu_covid  
12397            NaN  
12398            NaN  
12399            NaN  
12400            NaN  
12401            NaN  
12402            NaN  
12403            NaN  
12404            NaN  
12405            NaN  
12406            NaN  
12407            NaN  
12408            NaN  
12409            NaN  
12410            NaN  
12411            NaN  
12412            NaN  
12413            NaN  
12414            NaN  
12415            NaN  
12416            NaN

In [11]:
transpose(new_df, deaths_df, ['deaths_new'])
new_df.tail(20)

date              state cases_new deaths_new beds beds_covid  \
12397  2022-02-13         Terengganu       297          0  NaN        NaN   
12398  2022-02-13  W.P. Kuala Lumpur      1001          0  NaN        NaN   
12399  2022-02-13        W.P. Labuan       109          0  NaN        NaN   
12400  2022-02-13     W.P. Putrajaya       105          0  NaN        NaN   
12401  2022-02-14              Johor      2800          5  NaN        NaN   
12402  2022-02-14              Kedah      2733          3  NaN        NaN   
12403  2022-02-14           Kelantan      1330          2  NaN        NaN   
12404  2022-02-14             Melaka       666          0  NaN        NaN   
12405  2022-02-14    Negeri Sembilan       707          2  NaN        NaN   
12406  2022-02-14             Pahang       938          0  NaN        NaN   
12407  2022-02-14              Perak       461          1  NaN        NaN   
12408  2022-02-14             Perlis       129          0  NaN        NaN   
12409  2022-02-14       Pulau Pinang      1656          0  NaN        NaN   
12410  2022-02-14              Sabah      4035          5  NaN        NaN   
12411  2022-02-14            Sarawak       206          0  NaN        NaN   
12412  2022-02-14           Selangor      3955          4  NaN        NaN   
12413  2022-02-14         Terengganu       376          2  NaN        NaN   
12414  2022-02-14  W.P. Kuala Lumpur      1010          0  NaN        NaN   
12415  2022-02-14        W.P. Labuan       189          0  NaN        NaN   
12416  2022-02-14     W.P. Putrajaya       124          0  NaN        NaN   

      beds_icu_covid  
12397            NaN  
12398            NaN  
12399            NaN  
12400            NaN  
12401            NaN  
12402            NaN  
12403            NaN  
12404            NaN  
12405            NaN  
12406            NaN  
12407            NaN  
12408            NaN  
12409            NaN  
12410            NaN  
12411            NaN  
12412            NaN  
12413            NaN  
12414            NaN  
12415            NaN  
12416            NaN

In [12]:
transpose(new_df, beds_df, ['beds', 'beds_covid', 'beds_icu_covid'])
new_df.tail(20)

date              state cases_new deaths_new  beds beds_covid  \
12397  2022-02-13         Terengganu       297          0  1593        401   
12398  2022-02-13  W.P. Kuala Lumpur      1001          0  3963        572   
12399  2022-02-13        W.P. Labuan       109          0   202         98   
12400  2022-02-13     W.P. Putrajaya       105          0   609         56   
12401  2022-02-14              Johor      2800          5  5209       1378   
12402  2022-02-14              Kedah      2733          3  2664       1259   
12403  2022-02-14           Kelantan      1330          2  2684        362   
12404  2022-02-14             Melaka       666          0  1315        651   
12405  2022-02-14    Negeri Sembilan       707          2  1749       1084   
12406  2022-02-14             Pahang       938          0  1992        655   
12407  2022-02-14              Perak       461          1  5077        586   
12408  2022-02-14             Perlis       129          0   503         86   
12409  2022-02-14       Pulau Pinang      1656          0  2002        562   
12410  2022-02-14              Sabah      4035          5  4293       1469   
12411  2022-02-14            Sarawak       206          0  3571        678   
12412  2022-02-14           Selangor      3955          4  5378       1053   
12413  2022-02-14         Terengganu       376          2  1593        401   
12414  2022-02-14  W.P. Kuala Lumpur      1010          0  3963        572   
12415  2022-02-14        W.P. Labuan       189          0   202         98   
12416  2022-02-14     W.P. Putrajaya       124          0   609         56   

      beds_icu_covid  
12397             40  
12398             22  
12399              7  
12400              6  
12401             46  
12402            130  
12403             36  
12404             24  
12405             61  
12406             81  
12407             55  
12408             11  
12409             42  
12410            122  
12411             87  
12412             74  
12413             40  
12414             22  
12415              7  
12416              6

In [13]:
new_df.dtypes

date              object
state             object
cases_new         object
deaths_new        object
beds              object
beds_covid        object
beds_icu_covid    object
dtype: object

In [14]:
new_df[["cases_new", "deaths_new", "beds", "beds_covid", "beds_icu_covid"]] = new_df[["cases_new", "deaths_new", "beds", "beds_covid", "beds_icu_covid"]].apply(pd.to_numeric)
new_df.dtypes

date               object
state              object
cases_new         float64
deaths_new        float64
beds              float64
beds_covid        float64
beds_icu_covid    float64
dtype: object

In [15]:
new_df=new_df.sort_values(['date']).reset_index(drop=True)
new_df["cum_cases"]=new_df.groupby(['state'])['cases_new'].cumsum(axis=0)
new_df["cum_deaths"]=new_df.groupby(['state'])['deaths_new'].cumsum(axis=0)
new_df.sort_values(['date', 'state'])
new_df.tail(20)

date              state  cases_new  deaths_new    beds  \
12396  2022-02-13           Kelantan     1208.0         1.0  2684.0   
12397  2022-02-13              Kedah     1841.0         0.0  2664.0   
12398  2022-02-13              Johor     2419.0         4.0  5209.0   
12399  2022-02-13             Perlis      129.0         0.0   503.0   
12400  2022-02-14  W.P. Kuala Lumpur     1010.0         0.0  3963.0   
12401  2022-02-14         Terengganu      376.0         2.0  1593.0   
12402  2022-02-14           Selangor     3955.0         4.0  5378.0   
12403  2022-02-14            Sarawak      206.0         0.0  3571.0   
12404  2022-02-14              Sabah     4035.0         5.0  4293.0   
12405  2022-02-14       Pulau Pinang     1656.0         0.0  2002.0   
12406  2022-02-14             Perlis      129.0         0.0   503.0   
12407  2022-02-14    Negeri Sembilan      707.0         2.0  1749.0   
12408  2022-02-14             Pahang      938.0         0.0  1992.0   
12409  2022-02-14             Melaka      666.0         0.0  1315.0   
12410  2022-02-14           Kelantan     1330.0         2.0  2684.0   
12411  2022-02-14              Kedah     2733.0         3.0  2664.0   
12412  2022-02-14              Johor     2800.0         5.0  5209.0   
12413  2022-02-14        W.P. Labuan      189.0         0.0   202.0   
12414  2022-02-14              Perak      461.0         1.0  5077.0   
12415  2022-02-14     W.P. Putrajaya      124.0         0.0   609.0   

       beds_covid  beds_icu_covid  cum_cases  cum_deaths  
12396       362.0            36.0   189479.0      1287.0  
12397      1259.0           130.0   193132.0      2199.0  
12398      1320.0            46.0   282427.0      3943.0  
12399        86.0            11.0     8782.0       139.0  
12400       572.0            22.0   231117.0      2704.0  
12401       401.0            40.0    88491.0       766.0  
12402      1053.0            74.0   859521.0     10077.0  
12403       678.0            87.0   254433.0      1622.0  
12404      1469.0           122.0   278295.0      2856.0  
12405       562.0            42.0   180673.0      1775.0  
12406        86.0            11.0     8911.0       139.0  
12407      1084.0            61.0   127340.0      1344.0  
12408       655.0            81.0   110913.0       815.0  
12409       651.0            24.0    88302.0       977.0  
12410       362.0            36.0   190809.0      1289.0  
12411      1259.0           130.0   195865.0      2202.0  
12412      1378.0            46.0   285227.0      3948.0  
12413        98.0             7.0    11883.0       151.0  
12414       586.0            55.0   137946.0      1462.0  
12415        56.0             6.0    11824.0        22.0

In [16]:
new_df['state'] = new_df['state'].replace(['Pulau Pinang'],'Penang')
new_df['state'] = new_df['state'].replace(['W.P. Kuala Lumpur'],'Kuala Lumpur')
new_df['state'] = new_df['state'].replace(['W.P. Labuan'],'Labuan')
new_df['state'] = new_df['state'].replace(['W.P. Putrajaya'],'Putrajaya')
new_df['state'] = new_df['state'].replace(['Melaka'],'Malacca')
new_df.tail(20)

date            state  cases_new  deaths_new    beds  beds_covid  \
12396  2022-02-13         Kelantan     1208.0         1.0  2684.0       362.0   
12397  2022-02-13            Kedah     1841.0         0.0  2664.0      1259.0   
12398  2022-02-13            Johor     2419.0         4.0  5209.0      1320.0   
12399  2022-02-13           Perlis      129.0         0.0   503.0        86.0   
12400  2022-02-14     Kuala Lumpur     1010.0         0.0  3963.0       572.0   
12401  2022-02-14       Terengganu      376.0         2.0  1593.0       401.0   
12402  2022-02-14         Selangor     3955.0         4.0  5378.0      1053.0   
12403  2022-02-14          Sarawak      206.0         0.0  3571.0       678.0   
12404  2022-02-14            Sabah     4035.0         5.0  4293.0      1469.0   
12405  2022-02-14           Penang     1656.0         0.0  2002.0       562.0   
12406  2022-02-14           Perlis      129.0         0.0   503.0        86.0   
12407  2022-02-14  Negeri Sembilan      707.0         2.0  1749.0      1084.0   
12408  2022-02-14           Pahang      938.0         0.0  1992.0       655.0   
12409  2022-02-14          Malacca      666.0         0.0  1315.0       651.0   
12410  2022-02-14         Kelantan     1330.0         2.0  2684.0       362.0   
12411  2022-02-14            Kedah     2733.0         3.0  2664.0      1259.0   
12412  2022-02-14            Johor     2800.0         5.0  5209.0      1378.0   
12413  2022-02-14           Labuan      189.0         0.0   202.0        98.0   
12414  2022-02-14            Perak      461.0         1.0  5077.0       586.0   
12415  2022-02-14        Putrajaya      124.0         0.0   609.0        56.0   

       beds_icu_covid  cum_cases  cum_deaths  
12396            36.0   189479.0      1287.0  
12397           130.0   193132.0      2199.0  
12398            46.0   282427.0      3943.0  
12399            11.0     8782.0       139.0  
12400            22.0   231117.0      2704.0  
12401            40.0    88491.0       766.0  
12402            74.0   859521.0     10077.0  
12403            87.0   254433.0      1622.0  
12404           122.0   278295.0      2856.0  
12405            42.0   180673.0      1775.0  
12406            11.0     8911.0       139.0  
12407            61.0   127340.0      1344.0  
12408            81.0   110913.0       815.0  
12409            24.0    88302.0       977.0  
12410            36.0   190809.0      1289.0  
12411           130.0   195865.0      2202.0  
12412            46.0   285227.0      3948.0  
12413             7.0    11883.0       151.0  
12414            55.0   137946.0      1462.0  
12415             6.0    11824.0        22.0

In [17]:
new_df.to_csv("daily.csv", index=False)